# 1. Import and Install Dependencies



In [ ]:
!pip install -q langchain==0.0.219

!pip install -q transformers==4.29.2 sentencepiece==0.1.99 accelerate==0.19.0 bitsandbytes==0.39.0

!pip install -q python-dotenv==1.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 58.7 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch
from langchain.llms import HuggingFacePipeline
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

# 2. HuggingFace Model

In [ ]:
login(userdata.get('HF_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_id,device_map='auto')

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
pipe = pipeline("text2text-generation",
                model=base_model,
                tokenizer=tokenizer,
                max_length=256,
                temperature=0.4,
                top_p=0.95,
                repetition_penalty=1.2
                )

llm = HuggingFacePipeline(pipeline=pipe)

# 3. Generate Synonyms

In [ ]:
demo_message = """
Task: Rearrange the following scrambled words into a coherent sentence and complete it if necessary.

Example 1:
Scrambled Words: [love - I - dogs]
Complete the sentence:
I love dogs.

Example 2:
Scrambled Words: [beautiful - is - today - weather - the]
Complete the sentence:
The weather is beautiful today.

Example 3:
Scrambled Words: [eat - dinner - we - can - together]
Complete the sentence:
We can eat dinner together.

New Instance:
Scrambled Words: [Ischool - go - can - i]
Complete the sentence:
""".strip()




print( llm( demo_message ) )

I can go to ischool.


In [ ]:
demo_template = PromptTemplate(
    template="Task: Rearrange the following scrambled words into a coherent sentence and not complete it. Scrambled Words: {words} | Complete the sentence: {sentence}",
    input_variables=['words', 'sentence']
)

examples = [
    {"words": "[love - I - dogs]", "sentence": "I love dogs."},
    {"words": "[beautiful - is - today - weather - the]", "sentence": "The weather is beautiful today."},
    {"words": "[eat - dinner - we - can - together]", "sentence": "We can eat dinner together."},
]

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=demo_template,
    suffix="Scrambled Words: {words} |",
    input_variables=['words']
)

In [ ]:
print( llm( prompt.format(words='[school - go - can]') ) )

Complete the sentence: Can go to school


In [ ]:
print( llm( prompt.format(words='[sleep - want]') ) )

Complete the sentence: I want to sleep.
